# Ostomachion

In [ ]:
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
import matplotlib.patches 
from matplotlib.patches import FancyArrowPatch

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from collections import defaultdict

import io
from PIL import Image
import cairosvg
import subprocess
import pickle
import copy
import sys
import re

## Linked-list functions

In [ ]:
def find_center(ll):
    center = np.zeros(2)

    node = ll.head
    for i in range(ll.size):
        center += node.position
        node = node.next

    return center/ll.size

def scale_from_point(ll, scale, point):
    node = ll.head
    for i in range(ll.size):
        node.position = (node.position - point)*scale + point
        node = node.next


def find_point_equidistant(node, d):
    u = node.position - node.prev.position
    du = np.array([u[1], -u[0]])
    du = du/np.linalg.norm(du)

    v = node.position - node.next.position
    dv = np.array([-v[1], v[0]])
    dv = dv/np.linalg.norm(dv)
    
    return node.position + d*(v/(v@du) + u/(u@dv))
    
def find_intersection(A, B, P, Q):
    # AB defines a line, CD defines another line
    a = B-A
    p = Q-P
    ap = np.array([a[1], -a[0]])
    pp = np.array([p[1], -p[0]])

    s = ((A-P)@ap)/(p@ap)

    return P + p*s


def find_coincident(ll1, ll2, tol = 1e-3):
        
    pnode = ll1.head
    coincident = []
    for i in range(ll1.size):
        
        fnode = ll2.head
        for j in range(ll2.size):
    
            if np.linalg.norm(fnode.position - pnode.position) < tol:
                coincident.append([pnode,fnode])
                break
            fnode = fnode.next
    
        pnode = pnode.next

    return coincident

def find_node_from_index(ll, index):
    node = ll.head
    for i in range(index):
        node = node.next
    return node

def separate_poly_from_frame(poly_ll, frame_ll, d, tol = 1e-3):
    coincident = find_coincident(poly_ll, frame_ll, tol)
    
    for pnode, fnode in coincident:
        #print("coincide:", pnode.position, fnode.position)
    
        Q = find_point_equidistant(fnode, d)
        P = pnode.position
        A = pnode.next.position
        B = pnode.next.next.position
        #print("Q",Q)
        
        if np.linalg.norm(P-Q) > tol:
            #print("Shrinking")
            C = find_intersection(A, B, P, Q)
            scale = np.linalg.norm(Q-C)/np.linalg.norm(P-C)
            if scale > 1:
                continue
            scale_from_point(poly_ll, scale, C)

        #break

In [ ]:
class Node:
    def __init__(self, position, prev_node=None, next_node=None, immut = None):
        self.position = copy.deepcopy(position)
        self.position_immutable = copy.deepcopy(immut)
        self.prev = prev_node
        self.next = next_node

class List:
    def __init__(self, points):
        # Construct the linked list from a list of points
        if not points:
            self.size = 0
            self.head = None
            return
            
        self.size = len(points)
        node_points = [Node(point, None, None, point) for point in points]
        
        N = self.size
        self.head = node_points[0]
        for i in range(N):
            p = (i-1+N)%N
            n = (i+1)%N
            node_points[i].prev = node_points[p]
            node_points[i].next = node_points[n]

    def to_array(self):
        node = self.head
        nodes = []
        for i in range(self.size):
            nodes.append(node.position)
            node = node.next

        return nodes


    def to_vertices(self):
        node = self.head
        nodes = []
        for i in range(self.size):
            nodes.append(node.position)
            node = node.next

        return nodes

    def print(self):
        node = self.head
        print("Printing list")
        for i in range(self.size):
            print(node.position)
            node = node.next

    
    def print_all(self):
        node = self.head
        print("Printing list")
        for i in range(self.size):
            print(node.position, node.position_immutable)
            node = node.next

    def copy(self):
        points = self.to_array()
        return List(points)

        
    def delete_node(self, node):
        node.next.prev = node.prev
        node.prev.next = node.next
        if node == self.head:
            self.head = node.next
        self.size -= 1
        
    def plot(self, ax):
        node = self.head
        for i in range(self.size):
            x1, y1 = node.position
            x2, y2 = node.next.position
            ax.plot([x1, x2], [y1, y2], '-', color='k')
            node = node.next


def merge_second_into_first(list1, node1_index, list2, node2_index, a = 0.05, b = 0.05):

    a = max(a, b)
    
    node1 = find_node_from_index(list1, node1_index)
    node2 = find_node_from_index(list2, node2_index)
    
    copied_node1 = Node(copy.deepcopy(node1.position), node1, node1.next, copy.deepcopy(node1.position_immutable))
    node1.next.prev = copied_node1
    node1.next = copied_node1
    list1.size += 1

    copied_node2 = Node(copy.deepcopy(node2.position), node2, node2.next, copy.deepcopy(node2.position_immutable))
    node2.next.prev = copied_node2
    node2.next = copied_node2
    list2.size += 1

    unit = copied_node1.next.position - copied_node1.position
    unit = unit/np.linalg.norm(unit)
    copied_node1.position += unit*a
    
    unit = node1.prev.position - node1.position
    unit = unit/np.linalg.norm(unit)
    node1.position += unit*a

    
    unit = copied_node2.next.position - copied_node2.position
    unit = unit/np.linalg.norm(unit)
    copied_node2.position += unit*b
    
    unit = node2.prev.position - node2.position
    unit = unit/np.linalg.norm(unit)
    node2.position += unit*b

    copied_node1.prev = node2
    node2.next = copied_node1

    node1.next = copied_node2
    copied_node2.prev = node1
    

    list1.size += list2.size
    list2.size = 0
    list2.head = None

def prune_once(ll, tol = 1e-1):
    if ll.size < 3:
        return False
    
    # Check for coinciding points
    node = ll.head
    prev = node.prev
    prev_position = prev.position_immutable
    for i in range(ll.size):

        pos = node.position_immutable

        if np.linalg.norm(pos - prev_position) < tol:
            ll.delete_node(node)
            prev.position = prev.position_immutable
            return True

        
        prev = node
        node = node.next
        
        prev_position = pos


    for i in range(ll.size):
        unit1 = node.position_immutable - node.prev.position_immutable
        unit1 = unit1/np.linalg.norm(unit1)
        unit2 = node.position_immutable - node.next.position_immutable
        unit2 = unit2/np.linalg.norm(unit2)
        if np.linalg.norm(unit1 - unit2) < tol:
            ll.delete_node(node)
            return True

        node = node.next
        
    return False

## Auxiliary functions

In [ ]:
def scale(polygon, factor):
    points = []
    for x, y in polygon:
        new_x = x*factor
        new_y = y*factor
        points.append([new_x, new_y])
    return points

def angle_add(angle1: NDArray[np.float64], angle2: NDArray[np.float64]):
    cos1, sin1 = angle1
    cos2, sin2 = angle2
    return np.array([cos1*cos2 - sin1*sin2, cos1*sin2 + cos2*sin1])

def angle_sub(angle1: NDArray[np.float64], angle2: NDArray[np.float64]):
    # angle1 - angle2. Same as add, but sin2 swaps sign
    cos1, sin1 = angle1
    cos2, sin2 = angle2
    return np.array([cos1*cos2 + sin1*sin2, -cos1*sin2 + cos2*sin1])

# def vector_sub(v1, v2):
#     return np.array([v1[0] - v2[0], v1[1] - v2[1]])
    
def remove_frame(ax):
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    ax.tick_params(
        left=False, right=False, 
        top=False, bottom=False, 
        labelleft=False, labelbottom=False)

def add_grid(ax, width, color):
    for i in range(1,12):
        ax.plot([0, 12], [i,i], lw = width, color=color)
        ax.plot([i,i], [0, 12], lw = width, color=color)


In [ ]:
class Vertex:
    def __init__(self, p, prev, nex):
        self.position   =  np.array([p[0], p[1]], dtype=float)
        self.angle_to   =  np.array([nex[0] - p[0], nex[1] - p[1]], dtype=float)
        self.angle_from = -np.array([p[0] - prev[0], p[1] - prev[1]], dtype=float)

        self.angle_to   /= np.linalg.norm(self.angle_to)
        self.angle_from /= np.linalg.norm(self.angle_from)

class Polygon:
    def __init__(self, points, index=-1, color="w"):
        self.build_poly(points, index)
        self.color = color

    def build_poly(self, points, index):
        self.group_index = index
        
        self.vertices = []
        

        N = len(points)
        
        for i in range(N):
            current = points[i]
            pre = points[(i-1+N)%N]
            nex = points[(i+1)%N]
            self.vertices.append(Vertex(current, pre, nex))

        self.vertices_immutable = copy.deepcopy(self.vertices)
        
    def set_color(self, color):
        self.color = color

    def translate(self, vector):
        for v in self.vertices:
            v.position += vector

    def move_to(self, vector):
        dr = vector - self.vertices[0].position
        for v in self.vertices:
            v.position += dr
            
    def rotate(self, angle, rotation_center = np.zeros(2)):
        cos, sin = angle
        for v in self.vertices:
            x, y = v.position - rotation_center

            new_x =  x*cos + y*sin
            new_y = -x*sin + y*cos

            v.position = np.array([new_x, new_y]) + rotation_center
            v.angle_to = angle_sub(v.angle_to, angle)
            v.angle_from = angle_sub(v.angle_from, angle)
            

    def to_point_array(self):
        return [v.position for v in self.vertices]
            
    def get_center(self):
        xc, yc = 0, 0
        N = len(self.vertices)
        for v in self.vertices:
            x, y = v.position
            
            xc += x
            yc += y
        
        xc /= N
        yc /= N
        return xc, yc



    def print(self):
        print("polygon:")
        for vertex in self.vertices:
            print(vertex.position, end=" ")

class ArrowDetails:
    def __init__(self, edgecolor='w', facecolor='k', same_as_polygon = False):
        self.same_as_polygon = same_as_polygon
        self.edgecolor = edgecolor
        self.facecolor = facecolor

class PolygonDetails:
    def __init__(self, linewidth = 0.0, alpha = 1.0, linecolor='k', 
                 override_facecolor = None, doubleline = False, 
                 arrows = None, zorder = 20, linecolor_same_as_face = False, 
                 marker = '', markersize = 1):
        self.linewidth = linewidth
        self.alpha = alpha
        self.linecolor = linecolor
        self.linecolor_same_as_face = linecolor_same_as_face
        self.override_facecolor = override_facecolor
        self.arrows = arrows
        self.doubleline = doubleline
        self.zorder = zorder
        self.marker = marker
        self.markersize = markersize
        


def plot_poly(ax, poly, dr = np.array([0.0, 0.0]), scale = 1, details = PolygonDetails()):

    points = [v.position*scale + dr for v in poly.vertices]
    if len(points)<3:
        return

    color = poly.color
    if details.override_facecolor != None:
        color = details.override_facecolor

    
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    
    xs.append(xs[0])
    ys.append(ys[0])
    
    if details.doubleline:
        ax.plot(xs, ys, lw = details.linewidth*2, color = 'w', zorder = details.zorder)
        
    polygon = matplotlib.patches.Polygon(points, closed=True, facecolor=color, 
                                         lw = 0.0, 
                                         alpha = details.alpha, 
                                         zorder = details.zorder+1)
    ax.add_patch(polygon)

    ax.plot(xs, ys, lw = details.linewidth, marker = details.marker, 
            markersize = details.markersize, color = details.linecolor, 
            zorder = details.zorder+2)

    arrows = details.arrows
    if arrows != None:
        edgecolor = arrows.edgecolor
        facecolor = arrows.facecolor
        if arrows.same_as_polygon:
            facecolor = poly.color
        for i in range(len(points)):
            p0 = points[i]
            p1 = points[(i + 1) % len(points)]
            if np.linalg.norm(p0-p1) < 0.9:
                continue
    
            # Midpoint of the edge
            mid = 0.5 * (p0 + p1)
            direction = p1 - p0
            norm = np.linalg.norm(direction)
            if norm == 0:
                continue
    
            # Scale the arrow to be shorter than the edge
            arrow_length = 0.8   # change to tweak arrow size
            unit = direction / norm
            start = mid
            end = mid + 0.5 * arrow_length * unit
    
            # Draw arrow from start to end
            arrow = FancyArrowPatch(start, end,
                                    arrowstyle='simple',
                                    mutation_scale=30,
                                    linewidth=1,
                                    edgecolor=edgecolor, 
                                    facecolor=facecolor, zorder = details.zorder+3)
            
            ax.add_patch(arrow)



def plot_line(ax, poly, dr = np.array([0.0, 0.0]), scale = 1, details = PolygonDetails()):

    points = [v.position*scale + dr for v in poly.vertices]
    if len(points)<3:
        return

    color = poly.color
    if details.override_facecolor != None:
        color = details.override_facecolor

    
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    
    if details.doubleline:
        ax.plot(xs, ys, lw = details.linewidth*2, color = 'w', zorder = details.zorder)

    linecolor = details.linecolor
    if details.linecolor_same_as_face:
        linecolor = color
        
    ax.plot(xs, ys, lw = details.linewidth, marker = details.marker, 
            markersize = details.markersize, color = linecolor, 
            zorder = details.zorder+2)

    arrows = details.arrows
    if arrows != None:
        edgecolor = arrows.edgecolor
        facecolor = arrows.facecolor
        if arrows.same_as_polygon:
            facecolor = poly.color
        for i in range(len(points)-1):
            p0 = points[i]
            p1 = points[(i + 1) % len(points)]
            if np.linalg.norm(p0-p1) < 0.5:
                continue
    
            # Midpoint of the edge
            mid = 0.5 * (p0 + p1)
            direction = p1 - p0
            norm = np.linalg.norm(direction)
            if norm == 0:
                continue
    
            # Scale the arrow to be shorter than the edge
            arrow_length = 0.8   # change to tweak arrow size
            unit = direction / norm
            start = mid
            end = mid + 0.5 * arrow_length * unit
    
            # Draw arrow from start to end
            arrow = FancyArrowPatch(start, end,
                                    arrowstyle='simple',
                                    mutation_scale=30,
                                    linewidth=1,
                                    edgecolor=edgecolor, 
                                    facecolor=facecolor, zorder = details.zorder+3)
            
            ax.add_patch(arrow)



# State

In [ ]:
class State:
    def __init__(self, api):
        self.API = api

        
        # cmap1 = plt.get_cmap('Pastel1')
        # cmap2 = plt.get_cmap('Set3')
        # self.colors = [cmap1(i) for i in range(cmap1.N)] + [cmap2(i) for i in range(cmap1.N)]

        cmap = plt.get_cmap('tab20')
        self.colors = [cmap(i) for i in range(20)]

        
        # cmap = plt.get_cmap('Spectral')
        # self.colors = [cmap(i/13) for i in range(13)]
        
        self.frame = []
        self.polys = []
        self.history = []

    def set(self, frame, polys, history):
        self.frame = frame
        self.polys = polys
        self.history = history
        
    def set_from_history(self, history):
        state = self.API.state_from_history(history)
        self.frame = state.frame
        self.polys = state.polys
        self.history = history
        
        self.frame.set_color("w")
        for poly in self.polys:
            color = self.colors[poly.group_index]
            poly.set_color(color)
        
    def get_next_states(self):
        # Attribute colors to the polygons
        states_list = self.API.get_next_state(self)
        
        for state in states_list:    
            for poly in state.polys:
                color = self.colors[poly.group_index]
                poly.set_color(color)
                
        return states_list

    def print(self):
        print("History:", self.history)
        print("Frame:")
        for vertex in self.frame.vertices:
            print(vertex.position, end=" ")
        print("\nPolygons:")
        for poly in self.polys:
            for vertex in poly.vertices:
                print(vertex.position, end=" ")
            print("")

    # def plot(self, ax, include_frame = True):

    #     if include_frame:
    #         plot_poly(ax, self.frame, 1.0, 0.5)
        
    #     for poly in self.polys:
    #         plot_poly(ax, poly, 0.0, 1.0)

    
    def plot_in_position(self, ax, dr = np.zeros(2), scale = 1.0, 
                         frame_details = PolygonDetails(),
                         poly_details = PolygonDetails()):
        
        plot_poly(ax, self.frame, dr = dr, scale = scale, details = frame_details)
        
        for poly in self.polys:
            plot_poly(ax, poly, dr = dr, scale = scale, details = poly_details)
            
            

In [ ]:
def are_states_equal(state1, state2):
    groups1 = [[] for i in range(20)]
    groups2 = [[] for i in range(20)]
    
    for poly in state1.polys:
        groups1[poly.group_index].append(poly)

    for poly in state2.polys:
        groups2[poly.group_index].append(poly)

    for group1, group2 in zip(groups1, groups2):
        if len(group1) != len(group2):
            return False
        
        hash1 = 0
        for poly in group1:
            for vertex in poly.vertices:
                hash1 += vertex.position[0] + vertex.position[1]*142.3
                
        hash2 = 0
        for poly in group2:
            for vertex in poly.vertices:
                hash2 += vertex.position[0] + vertex.position[1]*142.3


        if abs(hash1 - hash2) > 1e-6:
            return False
            
    return True

# Extended state

In [ ]:
class ExtendedState:
    def __init__(self, state, idd):
        self.state = state
        self.id = idd
        self.children = []
        self.parent = None


def get_intermediate_exstates_from_history(api, history):
    
    # state = State(api)
    # state.set(frame2, [], [])
    state_list = []

    for i in range(len(history)//3+1):
        partial_history = history[:i*3]
        
        state = State(api)
        state.set_from_history(partial_history)
        state_list.append(ExtendedState(state, -1))

    return state_list

def attribute_ids(state_list):
    for i, state in enumerate(state_list):
        state.id = i

    
def connect_states(state_list):
    # Determine which state is children of which
    
    for i, state_i in enumerate(state_list):
        history_i = ",".join([str(p) for p in state_i.state.history])
        
        for j, state_j in enumerate(state_list):
            history_j = ",".join([str(p) for p in state_j.state.history])
    
            if history_i in history_j and len(state_i.state.history) == len(state_j.state.history) - 3:
                state_i.children.append(j)
                state_j.parent = i
    

def get_dict_equal_states(state_list):
    # Get a dictionary which maps each state to every state which is 
    # identical to it (including itself). The dictionary values get storted 
    # so that only the state with the lowest id remains out of identical states
    
    equals = defaultdict(list)
    for i, state_i in enumerate(state_list):
        for j, state_j in enumerate(state_list):
            if are_states_equal(state_i.state, state_j.state):
                equals[i].append(j)
                
    for key, vals in equals.items():
        vals = list(set(vals))
        vals = sorted(vals)
        equals[key] = vals
        
    return equals


def remove_repeated(state_list, equals):
    # If a state A is identical to some other B, A should be removed and its
    # references replaced by those of state B. 

    repeated = []
    for key, vals in equals.items():
        repeated += vals[1:]
    
    corrected_states = []
    for i, state in enumerate(state_list):
        
        if i in repeated:
            continue
    
        children = []
        for i in state.children:
            children.append(equals[i][0])
    
        children = list(set(children))
        state.children = children
        if state.parent:
            state.parent = equals[state.parent][0]
        corrected_states.append(state)
        
    return corrected_states

# API

In [ ]:

class API:
    def __init__(self, puzzle_type, selection_type, respect_restrictions):
        self.get_next_program = "../solver/build/interactive"
        self.state_fromh_program = "../solver/build/state_from_history"
        self.puzzle_type = puzzle_type
        self.selection_type = selection_type
        self.respect_restrictions = respect_restrictions

    def get_next_state(self, state):
        arguments = [self.get_next_program] + [self.puzzle_type, self.selection_type, self.respect_restrictions] + [str(index) for index in state.history]

        # Run the C++ program with arguments using subprocess
        result = subprocess.run(arguments, capture_output=True, text=True)
        # print(arguments)
        # print(result.stdout)
        states_list = self.parse_states_string(result.stdout)
        return states_list

    def state_from_history(self, history):
        arguments = [self.state_fromh_program] + [self.puzzle_type, self.selection_type, self.respect_restrictions] + [str(index) for index in history]

        # Run the C++ program with arguments using subprocess
        result = subprocess.run(arguments, capture_output=True, text=True)
        # print(arguments)
        # print(result.stdout)
        state = self.parse_states_string(result.stdout)[0]
        return state
    
    def parse_states_string(self, states_string):
    
        states = states_string.splitlines()
        states_list = []
    
        for state in states:
            
            if "History:" not in state:
                print("Restrictions were not respected")
                continue
    
            # Find the history
            b = state.split("{")[0].split(":")[1].split(" ")
            history = [int(i) for i in b if len(i) > 0]
    
            # Find the frame
            frame_points = []
            frame_loc = re.findall(r'\{([^\{]+)\}', state)
    
            if len(frame_loc) > 0:
                frame_txt = frame_loc[0]
                points = re.findall(r'\(([^\)]+)\)', frame_txt)
                for point in points:
                    x, y = point.split(",")
                    frame_points.append([float(x), float(y)])
    
            frame = Polygon(frame_points, -1)
    
            # Find all the used polygons
            polygon_groups = re.findall(r'<([^>]*)>', state)
            used_polygons_list = []
    
            for group_index, polygon_group in enumerate(polygon_groups):
                
                used_polygons = re.findall(r'\[([^\]]+)\]', polygon_group)
                for used_polygon in used_polygons:
                    
                    points = re.findall(r'\(([^\)]+)\)', used_polygon)
                    points_list = []
                    for point in points:
                        x, y = point.split(",")
                        points_list.append([float(x),float(y)])
    
                    used_polygons_list.append(Polygon(points_list, group_index))
    
            state = State(self)
            state.set(frame, used_polygons_list, history)
            states_list.append(state)

        return states_list



## GIF-making

In [ ]:
def io_to_png(io_frames):
    image_frames = []
    for buf in io_frames:
        png_bytes = cairosvg.svg2png(bytestring=buf.getvalue().encode('utf-8'))
        image = Image.open(io.BytesIO(png_bytes)).convert("RGBA")
        image_frames.append(image)

    return image_frames
    
    
def make_gif(gif_name, io_frames, duration):

    image_frames = io_to_png(io_frames)
    
    image_frames[0].save(gif_name,
       save_all=True,
       append_images=image_frames[1:],
       optimize=True,
       duration=duration,
       loop=0,
       disposal=2)

# Definitions

In [ ]:
frame2 = [[-6.0, -6.0], [-6.0, 6.0], [6.0, 6.0], [6.0, -6.0]]
frame2 = Polygon(frame2)
frame2.set_color("w")

# Original solution using OstomachionUnique, Leftest
og_u_history = [2,4,1,2,3,1,2,0,3,5,5,1,3,1,5,2,2,5,4,7,5,2,8,1,2,6,3,2,12,7,4,13,1,1,9,5,1,11,1,1,10,3]

# Nice solution using OstomachionUnique all
a_history = [3,4,2,0,13,5,6,11,3,3,10,3,2,3,2,0,6,2,0,8,3,9,12,3,4,1,0,2,7,0,0,2,8,3,0,0,3,9,3,0,5,4]
b_history = [2,6,1,2,4,3,4,10,3,4,12,1,4,1,3,8,2,2,2,3,3,2,0,1,2,11,0,2,13,2,2,7,3,2,9,2,2,8,2,1,5,4]

# Test

## Test: Linked list plotting capabilities

In [ ]:
api = API("OstomachionUnique", "leftest", "IgnoreRestrictions")
states = get_intermediate_exstates_from_history(api, og_u_history)
solution_state = states[-1].state
polys = solution_state.polys

In [ ]:
poly = polys[4]
poly.print()

arrows = ArrowDetails()
details = PolygonDetails(linewidth=2.0, linecolor = 'k', 
                         override_facecolor = 'y', arrows = arrows, doubleline = True)

fig, axs = plt.subplots(1,1,figsize=(5,5))
fig.patch.set_facecolor('blue')  # Figure background
axs.set_facecolor('blue')        # Plot area background

a = 6.2
axs.set_xlim([-a,a])
axs.set_ylim([-a,a])
remove_frame(axs)
plot_poly(axs, poly, details = details)
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(5,5))
a = 6.5
axs.set_xlim([-a,a])
axs.set_ylim([-a,a])
remove_frame(axs)

fig.patch.set_facecolor('blue')  # Figure background
axs.set_facecolor('blue')        # Plot area background

state = states[5].state

arrows = ArrowDetails()
frame_details = PolygonDetails(linewidth=2.0, linecolor = 'k', 
                         arrows = arrows, doubleline = True, zorder = 21, alpha=0.3)

poly_details = PolygonDetails(linewidth=0.0, linecolor = 'k', 
                         doubleline = False)

state.plot_in_position(axs, frame_details = frame_details, poly_details = poly_details)
plt.show()

## Test: using the engine to produce one state from the history

In [ ]:
api = API("OstomachionUnique", "all", "IgnoreRestrictions")
state = State(api)

# history = [1,5,4,4,9,2,0,2,3]
history = [2,6,2,2,1,2,5,7,1,5,5,3,5,4,2,8,8,1,8,9,0,3,0,0,5,3,2,6,2,0,7,12,3,1,11,5,2,13,3,1,10,1]
#history = [0,3,2,1,10,3,2,6,1]
state.set_from_history(history)

fig, ax = plt.subplots(1,1,figsize=(2,2))
a = 6.7
ax.set_xlim([-a, a])
ax.set_ylim([-a, a])
remove_frame(ax)

state.plot_in_position(ax)
plt.show()

state.print()
    
next_states = state.get_next_states()
print(len(next_states))

In [ ]:
api = API("OstomachionUnique", "leftest", "RespectRestrictions")
state = State(api)
state.set_from_history(a_history)

fig, ax = plt.subplots(1,1,figsize=(2,2))
a = 6.7
ax.set_xlim([-a, a])
ax.set_ylim([-a, a])
remove_frame(ax)

state.plot_in_position(ax)
plt.show()

In [ ]:
api = API("OstomachionUnique", "all", "RespectRestrictions")
state = State(api)
history = [3,4,2,0,13,5,6,11,3,3,10,3,2,3,2,0,6,2,0,8,3,9,12,3,4,1,0,2,7,0,0,2,8,3,0,0,3,9,3,0,5,4]
state.set_from_history(history)

fig, ax = plt.subplots(1,1,figsize=(2,2))
a = 6.7
ax.set_xlim([-a, a])
ax.set_ylim([-a, a])
remove_frame(ax)

state.plot_in_position(ax)
plt.show()

## Test: draw partial histories

In [ ]:
histories = []
histories.append([0,3,2,2,2,4,3,7,0])
histories.append([0,3,2,1,10,3,1,4,1])
histories.append([0,3,2,1,10,3,2,6,1])
histories.append([1,5,4,2,8,0,0,2,1])
histories.append([1,5,4,2,8,0,6,7,0])
histories.append([0,2,1,0,5,4,8,8,0])

In [ ]:
api = API("Ostomachion", "all", "RespectRestrictions")

state_list = []
for hh,history in enumerate(histories):
    state_list += get_intermediate_exstates_from_history(api, history)

attribute_ids(state_list)

fig, axs = plt.subplots(len(histories),4,figsize=(7,7))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

for n, state in enumerate(state_list):
    col = n%4
    row = n//4
    ax = axs[row, col] 
    ax.set_xlim([-6.1, 6.1])
    ax.set_ylim([-6.1, 6.1])
    remove_frame(ax)
        
    history_str = str(state.id) + "  -  " + ",".join([str(p) for p in state.state.history])
    ax.set_title(history_str, fontsize=5)
    state.state.plot_in_position(ax)
plt.show()

connect_states(state_list)
equals = get_dict_equal_states(state_list)
corrected_states = remove_repeated(state_list, equals)

In [ ]:
print(equals)
print("Complete state list")
for i, state in enumerate(state_list):
    print(state.id, state.parent, state.children)

print("Corrected state list")
for i, state in enumerate(corrected_states):
    print(state.id, state.parent, state.children)

In [ ]:
G = nx.DiGraph()

for state in corrected_states:
    for c in state.children:
        G.add_edge(state.id,c)

pos = graphviz_layout(G, prog='dot')

fig, ax = plt.subplots(figsize=(8, 7))
remove_frame(ax)

nx.draw_networkx_edges(G, pos, ax=ax, arrows=True, node_size=3600, node_shape='s', edge_color=(0.5, 0.5, 0.5))

no_frame = PolygonDetails(linewidth=0.0, zorder = 11, alpha=0.0)
thin_frame = PolygonDetails(linewidth=1.0, zorder = 11, alpha=0.0)
simple_poly = PolygonDetails(linewidth=0.0)

for state in corrected_states:
    coords = np.array(pos[state.id])

    scale = 4.0
    plot_poly(ax, frame2, dr = coords, scale = scale, details = thin_frame)
    state.state.plot_in_position(ax, coords, scale=scale, 
                                 frame_details = no_frame, poly_details = simple_poly)

plt.show()

# Load data

In [ ]:
def load_solutions(api, filename, produce_pickle = True):
    states_list = []
    with open(filename, "r") as f:
        lines = f.readlines()
        for ll, line in enumerate(lines):
            print(ll, end=" ")
            
            history = line.strip().split(" ")
            history = [int(digit) for digit in history]
            
            state = State(api)
            state.set_from_history(history)
            states_list.append(state)

    if produce_pickle:
        with open(filename.split(".")[0] + ".pickle", "wb") as f:
            pickle.dump(states_list, f)

    return states_list

def save_solutions_svg(states_list, filename):
    N = len(states_list)
    rows = int(N**0.5 - 0.01)+1
    print(N, "states", rows, "rows")
    
    fig, axs = plt.subplots(rows, rows, figsize=(rows*4, rows*4))
    
    for ax in axs.flatten():
        ax.set_xlim([-6.1, 6.1])
        ax.set_ylim([-6.1, 6.1])
        remove_frame(ax)
    
    for i, (ax, state) in enumerate(zip(axs.flatten(), states_list)):
        state.plot(ax)
        title = ",".join([str(i) for i in state.history])
        ax.set_title(title, fontsize=5)
    
    
    plt.savefig(filename, format="svg", transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
# Parameters
load_from_pickle = True
load_from_data = False
calculate_svg = False

### All solutions

In [ ]:
api = API("OstomachionUnique", "leftest", "RespectRestrictions")
filename = "out_ostomachion_unique3.txt"
if load_from_data:
    osto_states_unique = load_solutions(api, filename, produce_pickle = True)

In [ ]:
if load_from_pickle:
    with open("out_ostomachion_unique3.pickle", "rb") as f:
        osto_states_unique = pickle.load(f)

In [ ]:
if calculate_svg:
    save_solutions_svg(osto_states_unique, "ostomachion_solutions_u_history2.svg")

### All solutions not counting symmetries

In [ ]:
api = API("Ostomachion", "leftest", "RespectRestrictions")
filename = "out_ostomachion3.txt"

if load_from_data:
    osto_states = load_solutions(api, filename, produce_pickle = True)

In [ ]:
if load_from_pickle:
    with open("out_ostomachion3.pickle", "rb") as f:
        osto_states = pickle.load(f)

In [ ]:
if calculate_svg:
    save_solutions_svg(osto_states, "ostomachion_solutions_history2.svg")

# Common

In [ ]:
def find_index(history, states):
    for ss, state in enumerate(states):
        if state.history == history:
            return ss
    return -1

In [ ]:
# Indices for complete solutions - allow symmetrically identical
og = find_index([2,4,1,2,3,1,2,0,3,5,5,1,3,1,5,2,2,5,4,6,5,2,6,1,2,5,3,2,10,7,4,11,1,1,7,5,1,9,1,1,8,3], osto_states)
i1 = find_index([2,4,1,2,3,1,2,0,3,5,8,0,5,7,0,3,1,5,2,2,5,4,10,2,2,5,2,2,6,0,2,5,1,4,9,0,4,11,2,1,6,5], osto_states)
i2 = find_index([2,4,1,2,3,1,2,0,3,5,5,1,3,1,5,2,2,5,4,6,5,2,5,2,2,6,0,2,10,7,4,11,1,1,7,5,1,9,1,1,8,3], osto_states)
i3 = find_index([2,1,0,2,0,4,3,9,4,3,8,3,4,11,6,2,2,5,4,6,5,2,4,1,2,3,1,2,7,0,2,5,0,4,5,1,1,10,5,1,6,5], osto_states)
print(i1, i2, i3, og)

# Indices for complete solutions - ignore symmetrically identical
og_u = find_index([2,4,1,2,3,1,2,0,3,5,5,1,3,1,5,2,2,5,4,7,5,2,8,1,2,6,3,2,12,7,4,13,1,1,9,5,1,11,1,1,10,3], osto_states_unique)
i1_u = find_index([2,6,2,2,1,2,5,8,1,5,9,5,7,4,2,4,12,5,2,0,0,5,3,2,3,5,2,4,2,0,1,11,5,2,13,3,2,7,2,1,10,1], osto_states_unique)
i2_u = find_index([2,6,1,2,9,1,2,1,2,5,2,4,5,8,1,8,10,4,7,4,1,6,3,1,2,0,0,3,12,3,1,11,5,2,13,3,1,7,3,1,5,5], osto_states_unique)
i3_u = find_index([2,0,5,2,1,1,3,2,2,3,10,0,6,6,1,6,9,1,6,12,1,7,4,1,6,3,1,1,11,5,2,13,3,2,5,0,2,7,2,1,8,4], osto_states_unique)
print(i1_u, i2_u, i3_u, og_u)

In [ ]:
lw = 2
linecolor = 'k'

no_border = PolygonDetails(linewidth = 0.0)

no_border_transparent = PolygonDetails(linewidth = 0.0, 
                                       alpha = 0.0)

border = PolygonDetails(linewidth = lw, 
                        linecolor = linecolor)

border_transparent = PolygonDetails(linewidth = lw, 
                                    linecolor = linecolor, 
                                    alpha = 0.0)

dborder = PolygonDetails(linewidth = lw, 
                         linecolor = linecolor, 
                         doubleline = True,
                         zorder = 21)

arrows = ArrowDetails()
frame_details = PolygonDetails(linewidth = lw, 
                               linecolor = linecolor, 
                               arrows = arrows, 
                               zorder = 21, 
                               alpha=0.0, 
                               marker = 'o', 
                               markersize = 0)

frame_transparent_behind = PolygonDetails(linewidth = lw, 
                                          linecolor = linecolor, 
                                          zorder = 11, 
                                          alpha = 0.0)

arrow_poly = PolygonDetails(linewidth = lw, 
                            arrows = arrows)

hide = no_border_transparent

# Non-chapter related

## Ostomachion gif

In [ ]:
api = API("OstomachionUnique", "all", "RespectRestrictions")
history = a_history
states = get_intermediate_exstates_from_history(api, history)
state = states[-1].state
polys = state.polys

def update(ax, progress):
    remove_frame(ax)
    
    # Plot the frame and state
    states[0].state.plot_in_position(ax, frame_details = border_transparent)
    states[progress].state.plot_in_position(ax, frame_details = no_border_transparent)

frames = []
for progress in range(15):
    
    fig, ax = plt.subplots(1, 1, figsize=(5,5))
    update(ax, progress)

    # Convert to image and store to frames
    buf = io.StringIO()
    plt.savefig(buf, dpi=200, format="svg", transparent = True, bbox_inches="tight")
    frames.append(buf)

make_gif("common/ostomachion_solution.gif", frames, 500)

# plt.savefig(f"common/gif_frames/frame_{progress:03d}.svg", 
#                 dpi=200, format="svg", transparent = True, bbox_inches="tight")
        

# Chapter 1

## Original solution

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
remove_frame(ax)
ax.set_xlim([-6.1, 6.1])
ax.set_ylim([-6.1, 6.1])
osto_states_unique[og_u].plot_in_position(ax)
plt.savefig("chapter_1/c1a_og_solution.svg", format="svg", transparent=True, bbox_inches='tight')
plt.show()

## Some solutions

In [ ]:
indices = [i1_u, i2_u, i3_u]
N = len(indices)

fig, axs = plt.subplots(1, N, figsize=(5.5*N, 5))
for ii, index in enumerate(indices):
    ax = axs[ii]
    remove_frame(ax)
    ax.set_xlim([-6.1, 6.1])
    ax.set_ylim([-6.1, 6.1])
    osto_states_unique[index].plot_in_position(ax)

plt.savefig("chapter_1/c1b_3solutions.svg", format="svg", transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
# Separate images
indices = [i1_u, i2_u, i3_u]
N = len(indices)

for ii, index in enumerate(indices):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    
    remove_frame(ax)
    ax.set_xlim([-6.1, 6.1])
    ax.set_ylim([-6.1, 6.1])
    osto_states_unique[index].plot_in_position(ax)

    #plt.savefig(f"chapter_1/c1b_solutions{ii}.svg", format="svg", transparent=True, bbox_inches='tight')
    plt.show()

# Chapter 2

## Random triangle

In [ ]:
state = osto_states_unique[1]
poly = copy.deepcopy(state.polys[6])
poly.rotate([0.1,-0.9])
poly.translate(np.array([0.2, -0.2]))
    
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_xlim([-6.1, 6.1])
ax.set_ylim([-6.1, 6.1])
remove_frame(ax)

plot_poly(ax, poly, details = no_border)
plot_poly(ax, frame2, details = border_transparent)
plt.savefig("chapter_2/c2a_impossible_state.svg", format="svg", transparent=True, bbox_inches="tight")
plt.show()

## Polygon pool animation

In [ ]:
api = API("OstomachionUnique", "all", "RespectRestrictions")
history = a_history
states = get_intermediate_exstates_from_history(api, history)
state = states[-1].state
polys = state.polys

In [ ]:
def bring_poly_inside(lims_x, lims_y, poly):
    min_x, min_y, max_x, max_y = 99, 99, -99, -99
    for vertex in poly.vertices:
        x, y = vertex.position
        
        min_x = min(x, min_x)
        min_y = min(y, min_y)
        max_x = max(x, max_x)
        max_y = max(y, max_y)

    dleft  = min_x - lims_x[0]
    dright = max_x - lims_x[1]
    dtop   = max_y - lims_y[1]
    dbot   = min_y - lims_y[0]
    
    if dleft < 0:
        poly.translate(np.array([-dleft, 0]))

    if dright > 0:
        poly.translate(np.array([-dright, 0]))
    
    if dtop > 0:
        poly.translate(np.array([0, -dtop]))

    if dbot < 0:
        poly.translate(np.array([0, -dbot]))

exploded_polys = []
min_x, min_y, max_x, max_y = 0, 0, 0, 0
for polyc in polys:
    poly = copy.deepcopy(polyc)
    exploded_polys.append(poly)
    
    dx, dy = 0*(np.random.random(2)*2 - 1)
    poly.translate(np.array([dx, dy]))

    cos, sin = 2*np.random.random(2)-1
    norm = np.sqrt(cos*cos + sin*sin)
    cos /= norm
    sin /= norm
    poly.rotate(np.array([cos, sin]))

    bring_poly_inside([-10, 2], [-6, 6], poly)

    for v in poly.vertices:
        x, y = v.position
        min_x = min(x, min_x)
        min_y = min(y, min_y)
        max_x = max(x, max_x)
        max_y = max(y, max_y)


In [ ]:
frame_position = np.array([10.0, 0.0])

xlims = [min(frame_position[0] - 6, min_x) - 1, max(frame_position[0] + 6, max_x) + 1]
ylims = [min(frame_position[1] - 6, min_y) - 1, max(frame_position[1] + 6, max_y) + 1]


def update(ax, progress):
    ax.clear()
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    remove_frame(ax)
    
    polys_left = state.history[1+3*progress::3]
    
    for poly_index in polys_left[-1:0:-1]:
        poly = exploded_polys[poly_index]
        plot_poly(ax, poly, 0.0, 0.9)
    
    # Plot the frame and state
    #frame = copy.deepcopy(frame2)
    states[0].state.plot_in_position(ax, frame_position, frame_details = border_transparent)
    states[progress].state.plot_in_position(ax, frame_position, frame_details = no_border_transparent)
    
    if polys_left:
        
        # Highlight the next polygon
        next_poly = exploded_polys[polys_left[0]]
        plot_poly(ax, next_poly, details = dborder)
        
        probe_vertex = next_poly.vertices[0]
        px, py = probe_vertex.position
    
        frame_vertex_index = state.history[3*progress]
        anchor = states[progress].state.frame.vertices[frame_vertex_index].position + frame_position

        ax.scatter(px, py, color='w', zorder = 50, s = 100)
        ax.scatter(px, py, color='r', zorder = 50, s = 50)
        ax.scatter(*anchor, color='w', zorder = 50, s = 100)
        ax.scatter(*anchor, color='r', zorder = 50, s = 50)
        
        # Create a curved arrow
        arrow = FancyArrowPatch([px, py], anchor, connectionstyle="arc3,rad=0.3",
            arrowstyle='simple', linewidth=1.5, edgecolor='white', facecolor='black',
            mutation_scale=20, zorder = 200, shrinkA = 10, shrinkB = 10)
        ax.add_patch(arrow)

width = 8
height = width*(ylims[1] - ylims[0])/(xlims[1] - xlims[0])

frames = []
for progress in range(15):
    fig, ax = plt.subplots(1, 1, figsize=(width, height))
    update(ax, progress)
    buf = io.StringIO()
    plt.savefig(buf, dpi=200, format="svg", transparent = True, bbox_inches="tight")
    frames.append(buf)
    
make_gif("chapter_2/c2b_stop_motion_shapes.gif", frames, 500) 

## Configuration graph

In [ ]:
api = API("OstomachionUnique", "all", "RespectRestrictions")
histories = []
histories.append([0,3,2,2,2,4,3,7,0])
histories.append([0,3,2,1,10,3,1,4,1])
histories.append([0,3,2,1,10,3,2,6,1])
histories.append([1,5,4,2,8,0,0,2,1])
histories.append([1,5,4,2,8,0,6,7,0])
histories.append([0,2,1,0,5,4,8,8,0])

In [ ]:
state_list = []
for hh,history in enumerate(histories):
    state_list += get_intermediate_exstates_from_history(api, history)

attribute_ids(state_list)
connect_states(state_list)
equals = get_dict_equal_states(state_list)
corrected_states = remove_repeated(state_list, equals)

In [ ]:

fig, axs = plt.subplots(len(histories),4,figsize=(7,7))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

for n, state in enumerate(state_list):
    col = n%4
    row = n//4
    ax = axs[row, col] 
    ax.set_xlim([-6.1, 6.1])
    ax.set_ylim([-6.1, 6.1])
    remove_frame(ax)
        
    history_str = str(state.id) + "  -  " + ",".join([str(p) for p in state.state.history])
    ax.set_title(history_str)
    state.state.plot_in_position(ax, frame_details = border)
plt.show()

In [ ]:

G = nx.DiGraph()

for state in corrected_states:
    for c in state.children:
        G.add_edge(state.id,c)

pos = graphviz_layout(G, prog='dot')


fig, ax = plt.subplots(figsize=(8, 7))
remove_frame(ax)

nx.draw_networkx_edges(G, pos, ax=ax, arrows=True, node_size=3600, node_shape='s', edge_color=(0.5, 0.5, 0.5))

scale = 4.0
for state in corrected_states:
    coords = np.array(pos[state.id])
    corrected_states[0].state.plot_in_position(ax, coords, scale=scale, 
                                               frame_details = border_transparent)
    
    state.state.plot_in_position(ax, coords, scale=scale, 
                                 frame_details = no_border_transparent, poly_details = no_border)

plt.savefig("chapter_2/c2c_dag.svg", format="svg", transparent = True, bbox_inches='tight')
plt.show()


## Observation 2

In [ ]:
api = API("OstomachionUnique", "all", "RespectRestrictions")
history = [2,6,0,3,1,2]
states = get_intermediate_exstates_from_history(api, history)

fig, ax = plt.subplots(1,1, figsize=(3,3))
ax.set_xlim([-6.1,6.1])
ax.set_ylim([-6.1,6.1])
remove_frame(ax)
states[0].state.plot_in_position(ax, frame_details = border_transparent)
states[1].state.plot_in_position(ax, frame_details = no_border_transparent, 
                                 poly_details = no_border)

plt.savefig("chapter_2/c2d_one_poly.svg", format="svg", transparent = True, bbox_inches = "tight")
plt.show()

In [ ]:
api = API("OstomachionUnique", "all", "RespectRestrictions")
history = [2,6,0,3,1,2]
states = get_intermediate_exstates_from_history(api, history)

fig, ax = plt.subplots(1,1, figsize=(3,3))
ax.set_xlim([-6.1,6.1])
ax.set_ylim([-6.1,6.1])
remove_frame(ax)
states[0].state.plot_in_position(ax, frame_details = border_transparent)
states[2].state.plot_in_position(ax, frame_details = no_border_transparent, 
                                 poly_details = no_border)


plt.savefig("chapter_2/c2d_two_poly_good.svg", format="svg", transparent = True, bbox_inches = "tight")
plt.show()

In [ ]:
api = API("OstomachionUnique", "all", "RespectRestrictions")
history = [2,6,0,4,3,4]
states = get_intermediate_exstates_from_history(api, history)

fig, ax = plt.subplots(1,1, figsize=(3,3))
ax.set_xlim([-6.1,6.1])
ax.set_ylim([-6.1,6.1])
remove_frame(ax)
states[0].state.plot_in_position(ax, frame_details = border_transparent)
states[2].state.plot_in_position(ax, frame_details = no_border_transparent, 
                                 poly_details = no_border)

plt.savefig("chapter_2/c2d_two_poly_bad.svg", format="svg", transparent = True, bbox_inches = "tight")
plt.show()

# Chapter 3

## Intro

In [ ]:
api = API("OstomachionUnique", "all", "IgnoreRestrictions")
history = [2,6,2,2,1,2,5,7,1,5,5,3,5,4,2,8,8,1,8,9,0,3,0,0,5,3,2,6,2,0,7,12,3,1,11,5,2,13,3,1,10,1]
states = get_intermediate_exstates_from_history(api, history)
solution_state = states[-1].state
polys = solution_state.polys

In [ ]:
max_iterations = 3

a = 6.5
for iteration in range(max_iterations):
    
    fig, ax = plt.subplots(1,1,figsize=(5,5))
    ax.clear()
    ax.set_xlim([-a, a])
    ax.set_ylim([-a, a])
    remove_frame(ax)
    states[iteration].state.plot_in_position(ax, frame_details = border_transparent, 
                                            poly_details = hide)
    
    plt.savefig(f"chapter_3/c3a_frame{iteration}.svg", format="svg", transparent = True, bbox_inches = "tight")
    

## Merge and Prune

In [ ]:
api = API("OstomachionUnique", "all", "IgnoreRestrictions")
history = b_history
states = get_intermediate_exstates_from_history(api, history)
solution_state = states[-1].state
polys = solution_state.polys

In [ ]:
# Order of display:
# 1. just the frame
# 2. frame + unmerged polygon
# 3. frame + merged polygon
# 4+. frame + pruned polygon 
d = 0.4

fig, ax = plt.subplots(1,1,figsize=(6,6))
max_iterations = 200

a = 6.2
progress = 0
prune_progress = 0
stage = 0

frames = []
durations = []
for iteration in range(max_iterations):
    duration = 300
    if progress > 13:
        break
    
    ax.clear()
    ax.set_xlim([-a, a])
    ax.set_ylim([-a, a])
    remove_frame(ax)

    frame_vertex_index = history[3*progress]
    poly_index = history[3*progress + 1]
    frame_points = states[progress].state.frame.to_point_array()
    poly_points = polys[poly_index].to_point_array()
    frame_ll = List(frame_points)
    poly_ll = List(poly_points)
    
    separate_poly_from_frame(poly_ll, frame_ll, d)
    separated_poly_ll = poly_ll.copy()
    
    frame_node = find_node_from_index(frame_ll, frame_vertex_index)
    Q = find_point_equidistant(frame_node, d)
    P = frame_node.position
    p = P - frame_node.next.position
    p = p / np.linalg.norm(p)
    α = np.linalg.norm((P-Q)@p)

    print(iteration, stage, progress, prune_progress, frame_ll.size)
    if 0 <= stage < 1:
        # Plot just the frame
        
        temp_poly = Polygon(frame_ll.to_array())
        temp_poly.set_color("w")
        plot_poly(ax, temp_poly, details = frame_details)
        
        states[progress].state.plot_in_position(ax, frame_details = hide, poly_details = no_border)

    if 1 <= stage < 2:
        # Plot the frame and unmerged poly
        temp_poly = Polygon(poly_ll.to_array())
        temp_poly.set_color(polys[poly_index].color)
        plot_poly(ax, temp_poly, details = arrow_poly)

        temp_poly = Polygon(frame_ll.to_array())
        temp_poly.set_color("w")
        plot_poly(ax, temp_poly, details = frame_details)
        
        states[progress].state.plot_in_position(ax, frame_details = hide, poly_details = no_border)

    if stage >= 2:
        merge_second_into_first(frame_ll, frame_vertex_index, poly_ll, 0, a=α, b=d)
        
    if 2 <= stage < 3:
        duration = 700
        # Plot the frame and merged poly
        temp_poly = Polygon(separated_poly_ll.to_array())
        temp_poly.set_color(polys[poly_index].color)
        plot_poly(ax, temp_poly, details = no_border)

        temp_poly = Polygon(frame_ll.to_array())
        temp_poly.set_color("w")
        plot_poly(ax, temp_poly, details = frame_details)
        
        states[progress].state.plot_in_position(ax, frame_details = hide, poly_details = no_border)

    if 3 <= stage:
        
        modified = True
        for k in range(prune_progress):
            modified = prune_once(frame_ll, 1e-2)
            if not modified:
                break
       
        if modified:
            temp_poly = Polygon(separated_poly_ll.to_array())
            temp_poly.set_color(polys[poly_index].color)
            plot_poly(ax, temp_poly, details = no_border)
        
            temp_poly = Polygon(frame_ll.to_array())
            temp_poly.set_color("w")
            plot_poly(ax, temp_poly, details = frame_details)
            
            states[progress].state.plot_in_position(ax, frame_details = hide, poly_details = no_border)
            prune_progress += 1
            
        else:
            temp_poly = Polygon(frame_ll.to_array())
            temp_poly.set_color("w")
            plot_poly(ax, temp_poly, details = frame_details)
            
            states[progress+1].state.plot_in_position(ax, frame_details = hide, poly_details = no_border)
            progress += 1
            prune_progress = 0
            stage = -1

    stage += 1

    buf = io.StringIO()
    plt.savefig(buf, format="svg", transparent = True, bbox_inches="tight")
    frames.append(buf)
    durations.append(duration)

# Make the last frame last longer so you can see the final result clearly
durations[-1] = 1000
plt.show()

In [ ]:
image_frames = io_to_png(frames)

In [ ]:
for i in [0, 1, 2]:
    
    with open(f"chapter_3/c3b_mergeprune{i}.svg", "w", encoding="utf-8") as f:
        f.write(frames[i].getvalue())
        
    # Preview
    fig, ax = plt.subplots(figsize=(2,2))
    ax.imshow(image_frames[i])
    plt.show()

In [ ]:

for i in [9,11,15,18]:
    
    with open(f"chapter_3/c3c_mergeprune{i}.svg", "w", encoding="utf-8") as f:
        f.write(frames[i].getvalue())
        
    # Preview
    fig, ax = plt.subplots(figsize=(2,2))
    ax.imshow(image_frames[i])
    plt.show()

In [ ]:
make_gif("chapter_3/c3d_prune_algorithm.gif", frames, duration)

# Chapter 4: Overlap

## Edge intersection and point inside

In [ ]:

poly_details = PolygonDetails(linewidth = 0, alpha = 0.8)

square = Polygon([[0, 0], [3, 0], [3, 3], [0, 3]])
square.set_color("w")

triangle1 = Polygon([[0.5, 1], [2.4, 1.2], [1, 4]])
triangle1.set_color("C3")

triangle2 = Polygon([[2.5, 3.2], [3.2, 3.1], [3.1, 4]])
triangle2.set_color("C3")

triangle3 = Polygon([[0.1, 0.2], [2.2, 1.1], [2.9, 0.2]])
triangle3.set_color("C0")


fig, ax = plt.subplots(1,1, figsize=(5,6))
ax.set_xlim([-0.1,3.5])
ax.set_ylim([-0.1,4.2])
remove_frame(ax)

plot_poly(ax, square, details = frame_transparent_behind)
plot_poly(ax, triangle1, details = poly_details)
plot_poly(ax, triangle2, details = poly_details)
plot_poly(ax, triangle3, details = poly_details)

plt.savefig("chapter_4/c4a_edges_intersect.svg", format="svg", transparent=True, bbox_inches="tight")

plt.show()

## Vertices coinciding with edges

In [ ]:

poly_arrows = ArrowDetails(same_as_polygon=True)

arrows = ArrowDetails()
semi_transparent_arrows = PolygonDetails(linewidth = lw/2, 
                              linecolor = linecolor, 
                              zorder = 21, 
                              alpha = 0.8, 
                              arrows = poly_arrows)

frame_details = PolygonDetails(linewidth = lw, 
                               linecolor = 'k', 
                               zorder = 11, 
                               alpha = 0.0, 
                               arrows = arrows)


square = Polygon([[0, 0], [12, 0], [12, 4], [4, 4], [4, 8], [8,8], [8,12], [0, 12]])
square.set_color("w")

triangle1 = Polygon([[0, 6], [1, 12], [8, 10]])
triangle1.set_color("C0")

triangle2 = Polygon([[4, 6.8], [6, 8], [10, 4]])
triangle2.set_color("C3")

triangle3 = Polygon([[0.0, 2], [4, 4.8], [8, 4], [8, 0.0]])
triangle3.set_color("C3")


fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_xlim([-1,13])
ax.set_ylim([-1,13])
remove_frame(ax)

plot_poly(ax, square, details = frame_details)
plot_poly(ax, triangle1, details = semi_transparent_arrows)
plot_poly(ax, triangle2, details = semi_transparent_arrows)
plot_poly(ax, triangle3, details = semi_transparent_arrows)

plt.savefig("chapter_4/c4b_edges_vertex_intersect.svg", format="svg", transparent=True, bbox_inches="tight")

plt.show()

In [ ]:

square = Polygon([[0, 0], [12, 0], [12, 6], [6, 6], [6, 12], [0, 12]])
square.set_color("w")

triangle1 = Polygon([[2, 12], [5.6, 12], [0.0, 2]])
triangle1.set_color("C0")

triangle2 = Polygon([[6, 7.2], [6, 10], [11.6, 6], [8.8, 6]])
triangle2.set_color("C3")


fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_xlim([-1,13])
ax.set_ylim([-1,13])
remove_frame(ax)

plot_poly(ax, square, details = frame_details)
plot_poly(ax, triangle1, details = semi_transparent_arrows)
plot_poly(ax, triangle2, details = semi_transparent_arrows)


plt.savefig("chapter_4/c4c_edges_vertex_intersect.svg", format="svg", transparent=True, bbox_inches="tight")

plt.show()

In [ ]:
lw = 3.0
poly_arrows = ArrowDetails(same_as_polygon = True)
arrows = ArrowDetails()
poly_details = PolygonDetails(linewidth=lw*2, linecolor = 'k', doubleline = False, zorder = 21, 
                              alpha=1.0, arrows = poly_arrows, linecolor_same_as_face = True)
frame_details = PolygonDetails(linewidth=lw, linecolor = 'k', doubleline = True, 
                               zorder = 11, alpha=0.0, arrows = arrows)
        
    
points  = Polygon([[-4.0, 0.0], [2, 0.0], [8.0, 0.0], [14, 0.0], [20.0, 0.0], [26, 0.0]])
points.set_color("k")

points1 = Polygon([[0.0, 4.0], [2, 0.0], [4.0, 4.0]])
points1.set_color("C3")

points2 = Polygon([[2, -4.0], [4.0, 0.0], [6, -4.0]])
points2.set_color("C0")

points3 = Polygon([[8.0, 4.0], [8.0, 0.0], [10, -4.0]])
points3.set_color("C3")

points4 = Polygon([[12.4, 0.0], [16.0, 0.0], [18, -4.0]])
points4.set_color("C0")

points5 = Polygon([[18.8, 0.0], [22, 0.0], [24.0, 4.0]])
points5.set_color("C3")

fig, ax = plt.subplots(1,1, figsize=(10,2))
ax.set_xlim([-4, 28])
ax.set_ylim([-4.4, 4.4])
remove_frame(ax)

lw = 3.0
plot_line(ax, points, details = frame_details)
plot_line(ax, points1, details = poly_details)
plot_line(ax, points2, details = poly_details)
plot_line(ax, points3, details = poly_details)
plot_line(ax, points4, details = poly_details)
plot_line(ax, points5, details = poly_details)

plt.savefig("chapter_4/c4d_stretched.svg", format="svg", transparent=True, bbox_inches="tight")

plt.show()

## Vertex-vertex coincidence

In [ ]:

poly_arrows = ArrowDetails(same_as_polygon=True)
arrows = ArrowDetails()

frame_details = PolygonDetails(linewidth=3, linecolor = 'k', doubleline = True, 
                               zorder = 11, alpha=0.0, arrows = arrows)

square = Polygon([[0, 0], [5.2, 0.0], [6, 2], [6.8, 0.0],  [12, 0], [12, 6], 
                  [6, 6], [6, 12], [0, 12]])
square.set_color("w")

triangle1 = Polygon([[0.0, 12.0], [0.0, 0.0], [6, 12.0]])
triangle1.set_color("C0")

triangle3 = Polygon([[6, 2], [12.0, 0.0], [6,6], [0.0, 0.0]])
triangle3.set_color("C0")

triangle2 = Polygon([[6,6], [12.0, 6], [6, 12]])
triangle2.set_color("C3")


fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_xlim([-1,13])
ax.set_ylim([-1,13])
remove_frame(ax)

plot_poly(ax, square, details = frame_details)
plot_poly(ax, triangle1, details = semi_transparent_arrows)
plot_poly(ax, triangle2, details = semi_transparent_arrows)
plot_poly(ax, triangle3, details = semi_transparent_arrows)

plt.savefig("chapter_4/c4e_vertex_intersection.svg", format="svg", transparent=True, bbox_inches="tight")

plt.show()

In [ ]:

def draw_wedge(ax, poly, color):
    origin = poly.vertices[1].position
    
    angle_start = poly.vertices[2].position - origin
    angle_end = poly.vertices[0].position - origin
    

        
    u1 = angle_start / np.linalg.norm(angle_start)
    u2 = angle_end / np.linalg.norm(angle_end)
    

    # Compute angles of the vectors w.r.t. x-axis (in degrees)
    angle1 = np.degrees(np.arctan2(u1[1], u1[0]))
    angle2 = np.degrees(np.arctan2(u2[1], u2[0]))
    
    # Compute start angle and sweep angle (ensure counter-clockwise direction)
    start_angle = angle1
    sweep_angle = (angle2 - angle1) % 360
    
    # Draw a filled circular sector using Wedge
    arc_radius = 4 - sweep_angle/360
    wedge = matplotlib.patches.Wedge(center=origin, r=arc_radius,
                          theta1=start_angle, theta2=start_angle + sweep_angle,
                          facecolor=color, alpha=0.3)
    
    ax.add_patch(wedge)
    

In [ ]:
lw = 3.0
poly_arrows = ArrowDetails(same_as_polygon = True)
arrows = ArrowDetails()
poly_details = PolygonDetails(linewidth=lw*2, linecolor = 'k', doubleline = False, zorder = 21, 
                              alpha=1.0, arrows = poly_arrows, linecolor_same_as_face = True)
frame_details = PolygonDetails(linewidth=lw, linecolor = 'k', doubleline = True, 
                               zorder = 11, alpha=0.0, arrows = arrows)
        
    
fig, ax = plt.subplots(1,1, figsize=(8,3))
ax.set_xlim([-1, 36])
ax.set_ylim([-5, 5])
remove_frame(ax)

frame  = Polygon([[6, 0], [0,0], [0, -5]], color = "w")
poly = Polygon([[4, 3], [0,0], [4, -3]], color = "C3")
draw_wedge(ax, poly, 'C3')
plot_line(ax, poly, details = poly_details)
draw_wedge(ax, frame, 'g')
plot_line(ax, frame, details = frame_details)

new_origin = np.array([15.0, 0.0])
frame = Polygon([[-4, -4], [0,0], [5, -2]], color = "w")
poly = Polygon([[4, -4], [0,0], [0, -5]], color = "C0")
frame.translate(new_origin)
poly.translate(new_origin)
draw_wedge(ax, poly, 'C0')
plot_line(ax, poly, details = poly_details)
draw_wedge(ax, frame, 'g')
plot_line(ax, frame, details = frame_details)

new_origin = np.array([30.0, 0.0])
frame = Polygon([[-4, -4], [0,0], [5, -2]], color = "w")
poly = Polygon([[0, 5], [0,0], [4, 4]], color = "C3")
frame.translate(new_origin)
poly.translate(new_origin)
draw_wedge(ax, poly, 'C3')
plot_line(ax, poly, details = poly_details)
draw_wedge(ax, frame, 'g')
plot_line(ax, frame, details = frame_details)

plt.savefig("chapter_4/c4f_wedges.svg", format="svg", transparent=True, bbox_inches="tight")

plt.show()